In [2]:
import EOSgenerators.crust_EOS as crust
import EOSgenerators.RMF_EOS as RMF
import EOSgenerators.Polytrope_EOS as Polytrope
import EOSgenerators.Strangeon_EOS as Strangeon
from scipy.interpolate import interp1d
import TOVsolver.main as main
import matplotlib.pyplot as plt
import numpy as np
import math
from TOVsolver.unit import g_cm_3,dyn_cm_2,km,Msun,MeV,fm

In [4]:
data_BPS = np.loadtxt("data_k")

In [6]:
Pressure_BPS = data_BPS[:,0]
Density_BPS = data_BPS[:,1]

In [8]:
print(data_BPS)

[[9.539300e+18 1.031700e+04]
 [5.337900e+20 1.047100e+05]
 [2.324100e+22 1.045700e+06]
 ...
 [6.117452e+32 1.325753e+14]
 [6.166111e+32 1.333660e+14]
 [6.215158e+32 1.341614e+14]]


In [10]:
# Constants
n0 = 2.68e14  # g/cm^3
rho_min = 0.5 * n0  # 1.34e14 g/cm^3
rho_max = 1.1 * n0  # 2.948e14 g/cm^3
c_squared = 9e20  # cm^2/s^2 (c^2 in cgs units)

In [12]:
# Check maximum density in data
max_rho_data = np.max(Density_BPS)
print(f"Maximum rho in data: {max_rho_data:.2e} g/cm^3 (vs 0.5 n0 = {rho_min:.2e} g/cm^3)")

Maximum rho in data: 1.34e+14 g/cm^3 (vs 0.5 n0 = 1.34e+14 g/cm^3)


In [14]:
# Interpolate BPS data (and extrapolate since data < 0.5 n0)
interp_BPS = interp1d(Density_BPS, Pressure_BPS, kind='linear', fill_value="extrapolate")

In [16]:
# Get pressure at rho_min from BPS (extrapolated)
p_at_rho_min = interp_BPS(rho_min)

In [18]:
# Choose polytropic index Gamma (midpoint of [1.77, 3.23] for demonstration)
Gamma = np.random.uniform(1.77,3.23)

In [20]:
print(Gamma)

3.156035033672559


In [22]:
# Calculate K by matching at rho_min: p = K * rho^Gamma
K = p_at_rho_min / (rho_min ** Gamma)

In [24]:
# Define the EOS function
def EOS(rho):
    """
    Equation of State: returns pressure p (dyn/cm^2) for a given mass density rho (g/cm^3).
    """
    if rho < rho_min:
        return interp_BPS(rho)
    elif rho_min <= rho <= rho_max:
        return K * (rho ** Gamma)
    else:
        return np.nan  # Beyond 1.1 n0 is not specified

In [26]:
# Generate EOS in the requested range
rho_range = np.linspace(rho_min, rho_max, 100)
p_range =np.array( [EOS(rho) for rho in rho_range])

In [28]:
print(rho_range)

[1.34000000e+14 1.35624242e+14 1.37248485e+14 1.38872727e+14
 1.40496970e+14 1.42121212e+14 1.43745455e+14 1.45369697e+14
 1.46993939e+14 1.48618182e+14 1.50242424e+14 1.51866667e+14
 1.53490909e+14 1.55115152e+14 1.56739394e+14 1.58363636e+14
 1.59987879e+14 1.61612121e+14 1.63236364e+14 1.64860606e+14
 1.66484848e+14 1.68109091e+14 1.69733333e+14 1.71357576e+14
 1.72981818e+14 1.74606061e+14 1.76230303e+14 1.77854545e+14
 1.79478788e+14 1.81103030e+14 1.82727273e+14 1.84351515e+14
 1.85975758e+14 1.87600000e+14 1.89224242e+14 1.90848485e+14
 1.92472727e+14 1.94096970e+14 1.95721212e+14 1.97345455e+14
 1.98969697e+14 2.00593939e+14 2.02218182e+14 2.03842424e+14
 2.05466667e+14 2.07090909e+14 2.08715152e+14 2.10339394e+14
 2.11963636e+14 2.13587879e+14 2.15212121e+14 2.16836364e+14
 2.18460606e+14 2.20084848e+14 2.21709091e+14 2.23333333e+14
 2.24957576e+14 2.26581818e+14 2.28206061e+14 2.29830303e+14
 2.31454545e+14 2.33078788e+14 2.34703030e+14 2.36327273e+14
 2.37951515e+14 2.395757

In [30]:
print(p_range)

[6.20520554e+32 6.44570180e+32 6.69248872e+32 6.94565347e+32
 7.20528337e+32 7.47146593e+32 7.74428880e+32 8.02383979e+32
 8.31020685e+32 8.60347811e+32 8.90374184e+32 9.21108645e+32
 9.52560051e+32 9.84737274e+32 1.01764920e+33 1.05130473e+33
 1.08571278e+33 1.12088228e+33 1.15682217e+33 1.19354141e+33
 1.23104898e+33 1.26935385e+33 1.30846503e+33 1.34839152e+33
 1.38914237e+33 1.43072660e+33 1.47315326e+33 1.51643143e+33
 1.56057019e+33 1.60557861e+33 1.65146582e+33 1.69824093e+33
 1.74591306e+33 1.79449136e+33 1.84398499e+33 1.89440311e+33
 1.94575490e+33 1.99804956e+33 2.05129629e+33 2.10550430e+33
 2.16068283e+33 2.21684111e+33 2.27398840e+33 2.33213395e+33
 2.39128705e+33 2.45145698e+33 2.51265304e+33 2.57488454e+33
 2.63816079e+33 2.70249114e+33 2.76788493e+33 2.83435150e+33
 2.90190022e+33 2.97054048e+33 3.04028165e+33 3.11113314e+33
 3.18310436e+33 3.25620472e+33 3.33044365e+33 3.40583060e+33
 3.48237501e+33 3.56008635e+33 3.63897409e+33 3.71904771e+33
 3.80031670e+33 3.882790

In [36]:
data_polytropic_grok = np.column_stack((p_range,rho_range))
np.savetxt("data_poytropic_grok", data_polytropic_grok,  fmt="%.6e")

In [ ]:
# Example output
print(f"Gamma = {Gamma}")
print(f"K = {K:.2e} (in cgs units)")
print(f"Pressure at rho = 0.7 n0 ({0.7 * n0:.2e} g/cm^3): {EOS(0.7 * n0):.2e} dyn/cm^2")

In [34]:
data_speed_of_sound = np.loadtxt("data_speed_of_sound")
pressure_speed_of_sound = data_speed_of_sound[:,0]
density_speed_of_sound = data_speed_of_sound[:,1]

In [32]:
eps_total = np.hstack((Density_BPS,rho_range,density_speed_of_sound))
pres_total = np.hstack((Pressure_BPS, p_range,pressure_speed_of_sound))

NameError: name 'density_speed_of_sound' is not defined

In [ ]:
print(eps_total)
print(len(eps_total))

In [ ]:
print(pres_total)

In [ ]:
import matplotlib.pyplot as plt
# Plotting
plt.plot( eps_total/(1.7827e12), pres_total/(1.6022e33))
plt.xlabel('Density (MeV/fm³)')
plt.ylabel('Pressure (MeV/fm³)')
plt.title('Pressure vs Density')
plt.xscale("log")
plt.yscale("log")
plt.xlim([10**2,10**4])
plt.ylim([10**0,10**4])
plt.grid(True)
#plt.legend()
plt.show()

In [ ]:
EOS_FULL_GROK = np.column_stack((pres_total, eps_total))
np.savetxt("EOS_FULL_GROK", EOS_FULL_GROK,  fmt="%.6e")

In [ ]:
Full_EOS = np.loadtxt("EOS_FULL_GROK")
P = Full_EOS[:,0]*dyn_cm_2
D = Full_EOS[:,1]*g_cm_3

In [ ]:
MR_EOS = main.OutputMR("",D,P).T
MRT_EOS = main.OutputMRT("",D,P).T

In [ ]:
plt.plot(MR_EOS[1]/km,MR_EOS[0]/Msun)
plt.show()

In [ ]:
plt.plot(MRT_EOS[1]/Msun, MRT_EOS[2] ,'g-', lw=2,label ="MS1")
plt.ylim([0,3000])
plt.xlim([0.8,2])
plt.show()

In [ ]:
print(MR_EOS[1]/km)

In [ ]:
print(MR_EOS[0]/Msun)